In [2]:
!wget https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz
!tar -xzf nips12raw_str602.tgz

--2023-05-09 11:03:50--  https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz
Resolving cs.nyu.edu (cs.nyu.edu)... 216.165.22.203
Connecting to cs.nyu.edu (cs.nyu.edu)|216.165.22.203|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12851423 (12M) [application/x-gzip]
Saving to: ‘nips12raw_str602.tgz’

nips12raw_str602.tg 100%[===================>]  12.26M  6.23MB/s    in 2.0s    

2023-05-09 11:03:53 (6.23 MB/s) - ‘nips12raw_str602.tgz’ saved [12851423/12851423]



In [3]:
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
import os
import numpy as np
import pandas as pd

DATA_PATH = 'nipstxt/'
folders = ["nips{0:02}".format(i) for i in range(0,13)]
papers = []
for folder in folders:
  file_names = os.listdir(DATA_PATH + folder)
  for file_name in file_names:
    file_name = DATA_PATH + folder + '/' + file_name
    with open(file_name,encoding='utf-8',mode='r+',errors='ignore') as file:
      text = file.read()
      papers.append(text)



In [5]:
print(len(papers))
print(papers[100][:10])

1740
410 
NEURA


In [6]:
%%time
import nltk
import tqdm

stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_text(papers):
  normalized_papers = []
  for paper in tqdm.tqdm(papers):
    paper = paper.lower()
    paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
    paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
    paper_tokens = [token for token in paper_tokens if len(token) > 1]
    paper_tokens = [token for token in paper_tokens if token not in stop_words]
    paper_tokens = list(filter(None,paper_tokens))
    if paper_tokens:
      normalized_papers.append(paper_tokens)
  
  return normalized_papers


norm_papers = normalize_text(papers)

print(norm_papers[0])

100%|██████████| 1740/1740 [00:33<00:00, 51.20it/s]

['performance', 'measure', 'associative', 'memory', 'learn', 'forget', 'anthony', 'kuh', 'department', 'electrical', 'engineering', 'university', 'hawaii', 'manoa', 'honolulu', 'hi', 'abstract', 'recently', 'many', 'modification', 'mcculloch', 'pitt', 'model', 'proposed', 'learning', 'forgetting', 'occur', 'given', 'network', 'never', 'saturates', 'cease', 'function', 'effectively', 'due', 'overload', 'information', 'learning', 'update', 'con', 'tinue', 'indefinitely', 'network', 'need', 'introduce', 'performance', 'measure', 'addi', 'tion', 'information', 'capacity', 'evaluate', 'different', 'network', 'mathematically', 'define', 'quantity', 'plasticity', 'network', 'efficacy', 'information', 'vector', 'probability', 'network', 'saturation', 'quantity', 'analytically', 'compare', 'different', 'network', 'introduction', 'work', 'ha', 'recently', 'undertaken', 'quantitatively', 'measure', 'computational', 'aspect', 'network', 'model', 'exhibit', 'attribute', 'neural', 'network', 'mccull

In [7]:
import gensim
#bigram = gensim.models.Phrases(norm_papers, min_count=20, threshold=20,delimiter=b'_') # putting delimiter is giving error.
bigram = gensim.models.Phrases(norm_papers, min_count=20, threshold=20) # higher threshold fewer phrases.
bigram_model = gensim.models.phrases.Phraser(bigram)


print(bigram_model[norm_papers[0][:100]])

['performance', 'measure', 'associative_memory', 'learn', 'forget', 'anthony', 'kuh', 'department_electrical', 'engineering_university', 'hawaii', 'manoa', 'honolulu', 'hi', 'abstract', 'recently', 'many', 'modification', 'mcculloch_pitt', 'model', 'proposed', 'learning', 'forgetting', 'occur', 'given', 'network', 'never', 'saturates', 'cease', 'function', 'effectively', 'due', 'overload', 'information', 'learning', 'update', 'con', 'tinue', 'indefinitely', 'network', 'need', 'introduce', 'performance', 'measure', 'addi', 'tion', 'information', 'capacity', 'evaluate', 'different', 'network', 'mathematically', 'define', 'quantity', 'plasticity', 'network', 'efficacy', 'information', 'vector', 'probability', 'network', 'saturation', 'quantity', 'analytically', 'compare', 'different', 'network', 'introduction', 'work', 'ha', 'recently', 'undertaken', 'quantitatively', 'measure', 'computational', 'aspect', 'network', 'model', 'exhibit', 'attribute', 'neural_network', 'mcculloch_pitt', 'mod

In [8]:
norm_corpus_bigram = [bigram_model[doc] for doc in norm_papers]
dictionary = gensim.corpora.Dictionary(norm_corpus_bigram)

print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, '1vs'), (1, '21ogn'), (2, '2elogn'), (3, '2logn'), (4, '2m'), (5, '4e'), (6, '4logn'), (7, '81ogn'), (8, '_0'), (9, '_5'), (10, 'a2'), (11, 'a2k'), (12, 'ability'), (13, 'able'), (14, 'abstract')]
Total Vocabulary Size: 78892


In [9]:
# Filter out words that occur less than 20 documents, or more than 60% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.6)
print('Total Vocabulary Size:', len(dictionary))

Total Vocabulary Size: 7756


In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigram]
print(bow_corpus[1][:50])

[(1, 1), (13, 1), (14, 5), (18, 1), (19, 1), (26, 1), (29, 2), (32, 1), (33, 1), (34, 2), (35, 1), (42, 1), (52, 1), (54, 1), (55, 1), (61, 1), (64, 11), (71, 2), (72, 3), (79, 1), (80, 1), (84, 1), (86, 1), (87, 1), (91, 4), (99, 1), (106, 1), (107, 1), (118, 8), (121, 3), (124, 1), (127, 10), (135, 1), (141, 1), (148, 1), (155, 1), (157, 1), (165, 1), (169, 1), (179, 3), (183, 2), (184, 1), (186, 1), (190, 2), (194, 1), (198, 5), (207, 1), (215, 1), (216, 3), (218, 2)]


In [11]:
print([(dictionary[idx] , freq) for idx, freq in bow_corpus[1][:50]])

[('a2', 1), ('adapt', 1), ('adaptive', 5), ('address', 1), ('adjusted', 1), ('allowed', 1), ('american_institute', 2), ('analytical', 1), ('analytically', 1), ('analyze', 2), ('analyzed', 1), ('approximate', 1), ('assume', 1), ('assumption', 1), ('asymptotic', 1), ('becomes', 1), ('behavior', 11), ('bound', 2), ('bounded', 3), ('called', 1), ('cannot', 1), ('cease', 1), ('certain', 1), ('change', 1), ('chosen', 4), ('collection', 1), ('compare', 1), ('complete', 1), ('condition', 8), ('conference', 3), ('considered', 1), ('constant', 10), ('corresponding', 1), ('curve', 1), ('defined', 1), ('depends', 1), ('derived', 1), ('developed', 1), ('difficult', 1), ('due', 3), ('earlier', 2), ('earliest', 1), ('easily', 1), ('eigenvalue', 2), ('energy', 1), ('equation', 5), ('examine', 1), ('expressed', 1), ('factor', 3), ('fig', 2)]


LDA Model

The Latent Dirichlet Allocation (LDA) technique is a generative probabilistic model in which each document is assumed to have a combination of topics similar to a probabilistic Latent Semantic Indexing model. In this case, the latent topics contain a Dirichlet prior over them.

K - Number of topics
N - number of words in the document 
M - number of docuemnts to analyse
Alpha - Dirichlet prior distribution parameter of per document topic distribution
Beta - Dirichlet prior of the per-topic word distribution
Phi(k) - word distribution for topic k
Theta(i) - topic distribution for document i
z(i,j) - topic assignment for w(i,j)
w(i,j) is the jth work in the ith document

Phi and Theta are dirichlet distribution, z ad w are multinomials.

In [13]:
%%time
TOTAL_TOPICS = 10
lda_model = gensim.models.LdaModel(corpus=bow_corpus, id2word=dictionary, chunksize=1740, 
                                   alpha='auto', eta='auto', random_state=42,
                                   iterations=500, num_topics=TOTAL_TOPICS, 
                                   passes=20, eval_every=None)

CPU times: user 1min 56s, sys: 2.33 s, total: 1min 58s
Wall time: 1min 57s


In [14]:
for topic_id, topic in lda_model.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

Topic #1:
0.018*"neuron" + 0.010*"signal" + 0.009*"spike" + 0.008*"pattern" + 0.007*"stimulus" + 0.006*"response" + 0.006*"noise" + 0.006*"frequency" + 0.006*"channel" + 0.005*"activity" + 0.005*"firing" + 0.005*"synaptic" + 0.004*"cell" + 0.004*"effect" + 0.004*"rate" + 0.004*"memory" + 0.004*"neural" + 0.004*"temporal" + 0.004*"unit" + 0.004*"threshold"

Topic #2:
0.019*"cell" + 0.010*"image" + 0.010*"response" + 0.010*"visual" + 0.009*"motion" + 0.008*"stimulus" + 0.006*"filter" + 0.006*"direction" + 0.006*"receptive_field" + 0.006*"spatial" + 0.006*"orientation" + 0.005*"unit" + 0.004*"signal" + 0.004*"neuron" + 0.004*"location" + 0.004*"layer" + 0.004*"field" + 0.004*"cortical" + 0.004*"contrast" + 0.004*"region"

Topic #3:
0.019*"image" + 0.014*"feature" + 0.009*"training" + 0.008*"recognition" + 0.008*"object" + 0.007*"word" + 0.006*"classification" + 0.005*"pattern" + 0.005*"classifier" + 0.004*"representation" + 0.004*"task" + 0.004*"trained" + 0.004*"vector" + 0.004*"class" +

In [16]:
topics = [[(term, round(wt, 3)) 
               for term, wt in lda_model.show_topic(n, topn=20)] 
                   for n in range(0, lda_model.num_topics)]

for idx, topic in enumerate(topics):
    print('Topic #'+str(idx+1)+':')
    print([term for term, wt in topic])
    print()

Topic #1:
['neuron', 'signal', 'spike', 'pattern', 'stimulus', 'response', 'noise', 'frequency', 'channel', 'activity', 'firing', 'synaptic', 'cell', 'effect', 'rate', 'memory', 'neural', 'temporal', 'unit', 'threshold']

Topic #2:
['cell', 'image', 'response', 'visual', 'motion', 'stimulus', 'filter', 'direction', 'receptive_field', 'spatial', 'orientation', 'unit', 'signal', 'neuron', 'location', 'layer', 'field', 'cortical', 'contrast', 'region']

Topic #3:
['image', 'feature', 'training', 'recognition', 'object', 'word', 'classification', 'pattern', 'classifier', 'representation', 'task', 'trained', 'vector', 'class', 'face', 'experiment', 'pixel', 'character', 'level', 'layer']

Topic #4:
['distribution', 'vector', 'approximation', 'linear', 'estimate', 'sample', 'variable', 'training', 'probability', 'matrix', 'class', 'gaussian', 'solution', 'let', 'equation', 'bound', 'theory', 'density', 'prior', 'optimal']

Topic #5:
['unit', 'rule', 'hidden_unit', 'net', 'training', 'pattern

In [18]:
topics_df = pd.DataFrame([[term for term, wt in topic] 
                              for topic in topics], 
                         columns = ['Term'+str(i) for i in range(1, 21)],
                         index=['Topic '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
Term1,neuron,cell,image,distribution,unit,node,neuron,state,state,neuron
Term2,signal,image,feature,vector,rule,class,circuit,training,action,activity
Term3,spike,response,training,approximation,hidden_unit,training,chip,dynamic,control,cell
Term4,pattern,visual,recognition,linear,net,tree,analog,vector,policy,map
Term5,stimulus,motion,object,estimate,training,classifier,bit,equation,step,motor
Term6,response,stimulus,word,sample,pattern,probability,current,sequence,task,pattern
Term7,noise,filter,classification,variable,task,expert,voltage,rate,reinforcement_learning,connection
Term8,frequency,direction,pattern,training,representation,classification,vector,nonlinear,optimal,control
Term9,channel,receptive_field,classifier,probability,activation,prediction,memory,hmm,environment,eye
Term10,activity,spatial,representation,matrix,structure,variable,state,signal,controller,movement


In [20]:
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic])  
                              for topic in topics],
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, lda_model.num_topics+1)]
                         )
topics_df

<ipython-input-20-8fa5e6084872>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Terms per Topic
Topic1,"neuron, signal, spike, pattern, stimulus, response, noise, frequency, channel, activity, firing, synaptic, cell, effect, rate, memory, neural, temporal, unit, threshold"
Topic2,"cell, image, response, visual, motion, stimulus, filter, direction, receptive_field, spatial, orientation, unit, signal, neuron, location, layer, field, cortical, contrast, region"
Topic3,"image, feature, training, recognition, object, word, classification, pattern, classifier, representation, task, trained, vector, class, face, experiment, pixel, character, level, layer"
Topic4,"distribution, vector, approximation, linear, estimate, sample, variable, training, probability, matrix, class, gaussian, solution, let, equation, bound, theory, density, prior, optimal"
Topic5,"unit, rule, hidden_unit, net, training, pattern, task, representation, activation, structure, sequence, architecture, connectionist, trained, layer, language, learn, connection, state, string"
Topic6,"node, class, training, tree, classifier, probability, expert, classification, prediction, variable, pattern, layer, unit, mixture, training_set, test, architecture, task, level, table"
Topic7,"neuron, circuit, chip, analog, bit, current, voltage, vector, memory, state, threshold, node, computation, size, layer, element, implementation, processor, connection, pattern"
Topic8,"state, training, dynamic, vector, equation, sequence, rate, nonlinear, hmm, signal, recurrent, linear, noise, matrix, step, convergence, gradient, word, trajectory, control"
Topic9,"state, action, control, policy, step, task, reinforcement_learning, optimal, environment, controller, robot, goal, reward, search, path, agent, td, solution, trial, cost"
Topic10,"neuron, activity, cell, map, motor, pattern, connection, control, eye, movement, behavior, dynamic, neural, response, oscillator, simulation, phase, unit, oscillation, et_al"
